In [1]:
def handling_empty_features(df, target, model, scaler=False):
    """
    Функция для автоматического прогнозирования недостающий значений. Принимает в себя датафрейм с недостающей переменной, 
    название этой переменной (таргета), модель и, если есть, метод стандартизации/нормализации данных
    Функция делит датафрейм на train и test и показывает R2 для них для анализа точности прогноза и переобучения.
    Функция возвращает заполненный спрогнозированными данными датафрейм
    """
    df['nan'], df.loc[df[target].isna(), 'nan'] = 1, 0
    train = df[df['nan'] == 1]
    test = df[df['nan'] == 0]
    train, test = train.drop(['nan'], axis=1), test.drop(['nan'], axis=1)
    y = train[target]
    X = train.drop(target, axis=1)
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=21)
    if scaler:
        X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
        X_valid_scaled = pd.DataFrame(scaler.transform(X_valid), columns=X_valid.columns)
        model.fit(X_train_scaled, y_train)
        y_train_preds = model.predict(X_train_scaled)
        y_valid_preds = model.predict(X_valid_scaled)
        evaluate_preds(y_train, y_train_preds, y_valid, y_valid_preds, R2=True)
        X_scaled = scaler.fit_transform(X)
        X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
        model.fit(X_scaled, y)
        y_train_preds = model.predict(X_scaled)
        test = test.drop(target, axis=1)
        test[target] = model.predict(test)
        test = test[[x for x in train.columns]]
        return pd.concat([train, test])
    else:
        model.fit(X_train, y_train)
        y_train_preds = model.predict(X_train)
        y_test_preds = model.predict(X_valid)
        evaluate_preds(y_train, y_train_preds, y_valid, y_test_preds, R2=True)
        model.fit(X, y)
        y_train_preds = model.predict(X)
        test = test.drop(target, axis=1)
        test[target] = model.predict(test)
        test = test[[x for x in train.columns]]
        return pd.concat([train, test])